In [1]:
import numpy as np
import re
from itertools import islice
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import scipy.constants as const

import os
import sys
from pathlib import Path
from matplotlib import style
from lmfit import Parameters, fit_report, minimize
from lmfit.models import LinearModel
style.use('default')                # needed for vscode dark theme to see the axis

save_as_pgf = 0
save_as_pdf = 1

if save_as_pgf == 1:
    #mpl.use("pgf")                        
    mpl.rc('text', usetex=True)           # needed for latex
    mpl.rc('pgf', texsystem='pdflatex')   # change this if using xetex or luatex
    mpl.rc('pgf', rcfonts=False)          # don't setup fonts from rc parameters
    mpl.rc('pgf', preamble="\n".join([    # load additional packages
           r"\usepackage[utf8x]{inputenc}",
           r"\usepackage[T1]{fontenc}",
           r"\usepackage{amsmath,amsfonts,amsthm,amssymb,amstext}"]))
    mpl.rc('font', size=8)                # default font size
    mpl.rc('font', family='serif')          # use serif fonts
    #mpl.rc('font', family='sans-serif')   # use sans-serif fonts
    mpl.rc('font', serif='Latin Modern')  # use latex default serif font
    mpl.rc('axes', titlesize=10)           # fontsize of the axes title
    mpl.rc('axes', titlepad=2)            # pad between axis and title
    mpl.rc('axes', labelsize=10)           # fontsize of the x and y labels
    mpl.rc('xtick', labelsize=8)          # fontsize of the tick labels
    mpl.rc('ytick', labelsize=8)          # fontsize of the tick labels
    mpl.rc('legend', fontsize=8)          # legend fontsize
    mpl.rc('figure', titlesize=10)         # fontsize of the figure title
    mpl.rc('lines', markersize=2)         # default line markersize
    mpl.rc('lines', marker="")            # default line marker
    mpl.rc('grid', alpha=0.4)             # transparency of the grid
    mpl.rc('grid', color="gray")          # color of the grid
    mpl.rc('grid', linewidth=0.5)         # linewidth of the grid
    mpl.rcParams['figure.constrained_layout.use'] = True # use constrained_layout
    mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=["k", "r", "b", "g", "orange", "purple"]) #matplotlib colororder-default ändern
    lw = 1.0                              # linewidth of the lines

    # width and height of tex document in inches
    # For Katrins Bachelor thesis
    # in Latex:
    # \usepackage{layouts}
    # textwidth in inches: \printinunitsof{in}\prntlen{\textwidth}
    
    fig_width = 6.202
    fig_height = fig_width * 0.6
    
elif save_as_pdf == 1:
    normal = 9                              # fontsize of the text
    mpl.rc('font', size=normal+2)             # default font size
    mpl.rc('font', family='serif')          # use serif fonts
    mpl.rc('axes', titlesize=normal)        # fontsize of the axes title
    mpl.rc('axes', titlepad=5)              # pad between axis and title
    mpl.rc('axes', labelpad=7)              # pad between axis and label
    mpl.rc('axes', labelsize=normal+2)      # fontsize of the x and y labels
    mpl.rc('xtick', labelsize=normal)       # fontsize of the tick labels
    mpl.rc('ytick', labelsize=normal)       # fontsize of the tick labels
    mpl.rc('legend', fontsize=normal)       # legend fontsize
    mpl.rc('figure', titlesize=normal+4)    # fontsize of the figure title
    mpl.rc('lines', markersize=12)          # default line markersize
    mpl.rc('lines', marker="")              # default line marker
    mpl.rc('grid', alpha=0.4)               # transparency of the grid              
    mpl.rc('grid', color="gray")            # color of the grid
    mpl.rc('grid', linewidth=0.5)           # linewidth of the grid
    mpl.rcParams["xtick.major.size"] = 8    # major tick size in points
    mpl.rcParams["ytick.major.size"] = 8    # major tick size in points
    mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=["k", "r", "b", "g", "orange", "purple"]) #matplotlib colororder-default ändern
    
    lw = 2
    ms_4 = 4
    ms_8 = 8

    # height and width for nice print in jupyter notebook
    fig_width = 8
    fig_height = 5
    
else:
    mpl.rc('font', size=16)                 # controls default text sizes
    mpl.rc('axes', titlesize=16)            # fontsize of the axes title
    mpl.rc('axes', labelsize=18)            # fontsize of the x and y labels
    mpl.rc('xtick', labelsize=16)           # fontsize of the tick labels
    mpl.rc('ytick', labelsize=16)           # fontsize of the tick labels
    mpl.rc('legend', fontsize=18)           # legend fontsize
    mpl.rc('figure', titlesize=18)          # fontsize of the figure title
    mpl.rc('grid', alpha=0.4)
    mpl.rc('lines', markersize=10)    
    
    

def save_plot_as_pgf(fig, path):
  if save_as_pgf == 1:
    fig.savefig(path, format="pgf")

def save_plot_as_pdf(fig, path):
  if save_as_pdf == 1:
    fig.savefig(path, format="pdf", bbox_inches='tight')


In [2]:
# load data: Haber_Bosch (HB)

with open('Haber_Bosch_E.copy', 'r') as f:
    lines = f.readlines()

HB = np.loadtxt(lines)
# NH3 plus kcal/mol factor
nh3_b2plyp = HB[0] * 627.503
nh3_mp2 = HB[1] * 627.503
nh3_b3lyp = HB[2] * 627.503
nh3_ccsdt = HB[3] * 627.503

#n2 plus kcal/mol factor
n2_b2plyp = HB[4] * 627.503
n2_mp2 = HB[5] * 627.503
n2_b3lyp = HB[6] * 627.503
n2_ccsdt = HB[7] * 627.503

#h2 plus kcal/mol factor
h2_b2plyp = HB[8] * 627.503
h2_mp2 = HB[9] * 627.503
h2_b3lyp = HB[10] * 627.503
h2_ccsdt = HB[11] * 627.503

#Enthalpy Corrections

with open('Enthalpy_correction-Copy1', 'r') as f:
    lines = f.readlines()
HB_thermo_corr = np.loadtxt(lines)
# NH3
nh3_b2plyp_corr = HB_thermo_corr[0]
nh3_mp2_corr = HB_thermo_corr[1]
nh3_b3lyp_corr = HB_thermo_corr[2]
nh3_ccsdt_corr = HB_thermo_corr[3]

#n2
n2_b2plyp_corr = HB_thermo_corr[4]
n2_mp2_corr = HB_thermo_corr[5]
n2_b3lyp_corr = HB_thermo_corr[6]
n2_ccsdt_corr = HB_thermo_corr[7]

#h2
h2_b2plyp_corr = HB_thermo_corr[8]
h2_mp2_corr = HB_thermo_corr[9]
h2_b3lyp_corr = HB_thermo_corr[10]
h2_ccsdt_corr = HB_thermo_corr[11]

#tpss E in kcal/mol
with open('TPSS_HB_E-Copy1', 'r') as f:
    lines = f.readlines()
HB_TPSS = np.loadtxt(lines)

#NH3
nh3_tpss = HB_TPSS[0] * 627.503
#N2
n2_tpss = HB_TPSS[1] * 627.503
#H2
h2_tpss = HB_TPSS[2] * 627.503

#tpss correction
with open('TPSS_Enthalpy_correction-Copy1', 'r') as f:
    lines = f.readlines()
HB_TPSS_corr = np.loadtxt(lines)
#NH3
nh3_tpss_corr = HB_TPSS_corr[0]
#N2
n2_tpss_corr = HB_TPSS_corr[1]
#H2
h2_tpss_corr = HB_TPSS_corr[2]
#ch4
ch4_tpss_corr = HB_TPSS_corr[3]
#co
co_tpss_corr = HB_TPSS_corr[4]
#h2o
h2o_tpss_corr = HB_TPSS_corr[5]
#calculate Reaction enthalpy = H(Products) - H (Educts) = 2 * H(Nh3) - H (N2) - 3 * H (h2) 

#b2plyp
H_reac_b2plyp = 2 * (nh3_b2plyp + nh3_b2plyp_corr) - 3 * (h2_b2plyp + h2_b2plyp_corr) - (n2_b2plyp + n2_b2plyp_corr)
#mp2
H_reac_mp2 = 2 * (nh3_mp2 + nh3_mp2_corr) - 3 * (h2_mp2 + h2_mp2_corr) - (n2_mp2 + n2_mp2_corr)
#b3lyp
H_reac_b3lyp = 2 * (nh3_b3lyp + nh3_b3lyp_corr) - 3 * (h2_b3lyp + h2_b3lyp_corr) - (n2_b3lyp + n2_b3lyp_corr)
#ccsd_T
H_reac_ccsdt = 2 * (nh3_ccsdt + nh3_ccsdt_corr) - 3 * (h2_ccsdt + h2_ccsdt_corr) - (n2_ccsdt + n2_ccsdt_corr)
#tpss
H_reac_tpss = 2 * (nh3_tpss + nh3_tpss_corr) - 3 * (h2_tpss + h2_tpss_corr) - (n2_tpss + n2_tpss_corr)
# load data: Steam reforming (SR)

with open('Steam_reforming_E-Copy1', 'r') as f:
    lines = f.readlines()

SR = np.loadtxt(lines)
# CH4 plus kcal/mol factor
ch4_b2plyp_SR = SR[0] * 627.503
ch4_mp2_SR = SR[1] * 627.503
ch4_b3lyp_SR = SR[2] * 627.503
ch4_ccsdt_SR = SR[3] * 627.503
ch4_tpss_SR = SR[4] * 627.503

#H2O plus kcal/mol factor
h2o_b2plyp_SR = SR[5] * 627.503
h2o_mp2_SR = SR[6] * 627.503
h2o_b3lyp_SR = SR[7] * 627.503
h2o_ccsdt_SR = SR[8] * 627.503
h2o_tpss_SR = SR[9] * 627.503

#CO plus kcal/mol factor
CO_b2plyp_SR = SR[10] * 627.503
CO_mp2_SR = SR[11] * 627.503
CO_b3lyp_SR = SR[12] * 627.503
CO_ccsdt_SR = SR[13] * 627.503
CO_tpss_SR = SR[14] * 627.503

#H2 plus kcal/mol factor siehe HB
#Enthalpy Corrections; tpss see HB

with open('Enthalpy_correction-Copy1', 'r') as f:
    lines = f.readlines()
SR_thermo_corr = np.loadtxt(lines)
# ch4
ch4_b2plyp_corr = SR_thermo_corr[12]
ch4_mp2_corr = SR_thermo_corr[13]
ch4_b3lyp_corr = SR_thermo_corr[14]
ch4_ccsdt_corr = SR_thermo_corr[15]

#co
co_b2plyp_corr = SR_thermo_corr[16]
co_mp2_corr = SR_thermo_corr[17]
co_b3lyp_corr = SR_thermo_corr[18]
co_ccsdt_corr = SR_thermo_corr[19]

#h2o
h2o_b2plyp_corr = SR_thermo_corr[20]
h2o_mp2_corr = SR_thermo_corr[21]
h2o_b3lyp_corr = SR_thermo_corr[22]
h2o_ccsdt_corr = SR_thermo_corr[23]

#calculate Reaction enthalpy = H(Products) - H (Educts) = H (CO) + 3 * H (H2) - H (H2O) - H (CH4) 

#b2plyp
SR_reac_b2plyp = (CO_b2plyp_SR + co_b2plyp_corr) + 3 * (h2_b2plyp + h2_b2plyp_corr) - (h2o_b2plyp_SR + h2o_b2plyp_corr) - (ch4_b2plyp_SR + ch4_b2plyp_corr) 
#mp2
SR_reac_mp2 = (CO_mp2_SR + co_mp2_corr) + 3 * (h2_mp2 + h2_mp2_corr) - (h2o_mp2_SR + h2o_mp2_corr) - (ch4_mp2_SR + ch4_mp2_corr) 
#b3lyp
SR_reac_b3lyp = (CO_b3lyp_SR + co_b3lyp_corr) + 3 * (h2_b3lyp + h2_b3lyp_corr) - (h2o_b3lyp_SR + h2o_b3lyp_corr) - (ch4_b3lyp_SR + ch4_b3lyp_corr) 
#ccsd_T
SR_reac_ccsdt = (CO_ccsdt_SR + co_ccsdt_corr) + 3 * (h2_ccsdt + h2_ccsdt_corr) - (h2o_ccsdt_SR + h2o_ccsdt_corr) - (ch4_ccsdt_SR + ch4_ccsdt_corr) 
#tpss
SR_reac_tpss = (CO_tpss_SR + co_tpss_corr) + 3 * (h2_tpss + h2_tpss_corr) - (h2o_tpss_SR + h2o_tpss_corr) - (ch4_tpss_SR + ch4_tpss_corr) 

#make table 

# convert to pandas dataframe and add column names
names = ['MP2', 'CCSD(T)', 'TPSS','b3lyp','b2plyp']
df = pd.DataFrame({'Haber-Bosch': [H_reac_mp2, H_reac_ccsdt,H_reac_tpss,H_reac_b3lyp, H_reac_b2plyp],'Steam reforming':[SR_reac_mp2,SR_reac_ccsdt,SR_reac_tpss,SR_reac_b3lyp,SR_reac_b2plyp]}, index=names)
df.columns = pd.MultiIndex.from_product([['Reaction Enthalpies'], df.columns])


# print dataframe
display(round(df,4))

#convert table to latex
print(df.to_latex(float_format="%.4f", escape=False))

Reaction Enthalpies                
                Haber-Bosch Steam reforming
MP2                -10.4916         41.7734
CCSD(T)            -22.8246         49.9673
TPSS               -12.6693         42.3520
b3lyp              -20.8464         47.2550
b2plyp             -22.8885         49.6725

\begin{tabular}{lrr}
\toprule
{} & \multicolumn{2}{l}{Reaction Enthalpies} \\
{} &         Haber-Bosch & Steam reforming \\
\midrule
MP2     &            -10.4916 &         41.7734 \\
CCSD(T) &            -22.8246 &         49.9673 \\
TPSS    &            -12.6693 &         42.3520 \\
b3lyp   &            -20.8464 &         47.2550 \\
b2plyp  &            -22.8885 &         49.6725 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_5557/950226000.py:164: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex(float_format="%.4f", escape=False))
